In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import scipy
import damselfly as df

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')

"""
Date: 6/25/2021
Description: Trained a dfcnn model on 210625_df_data1 data set. 
I trained and evaluated this model on with different noise temperatures.
Here I plot the results.
"""

# load the results

In [ ]:
result_dir = os.listdir(os.path.join(RESULTPATH, 'confusion_matrices'))[0]

In [ ]:
result_files = os.listdir(os.path.join(RESULTPATH, 'confusion_matrices', result_dir))

In [ ]:
def load_result(file):
    temp = float(file.split('temp')[-1].split('_')[0])
    dset = file.split('evaldata_')[-1].split('_')[0]
    threshold = float(file.split('threshold')[-1].split('.npz')[0])
    result = {}
    result_data = np.load(file)
    for i,key in enumerate(result_data):
        result[key] = result_data[key]
    return temp, dset, threshold, result

def load_all_results(result_files):
    all_results = []
    for file in result_files:
        temp, dset, threshold, result = load_result(os.path.join(RESULTPATH, 'confusion_matrices', result_dir, file))
        result_dict = {'T': temp, 'threshold': threshold, 'dset': dset }
        for i,key in enumerate(result):
            result_dict[key] = result[key]
        all_results.append(result_dict)
    return pd.DataFrame(all_results)

all_results = load_all_results(result_files)

# Get number of ground truth negatives and positives for each dataset

In [ ]:
test_dataset = os.listdir(DATAPATH)[0]

In [ ]:
testdata = df.data.DFDataset(os.path.join(DATAPATH, test_dataset), 'test')
traindata = df.data.DFDataset(os.path.join(DATAPATH, test_dataset), 'train')

In [ ]:
Nsignal_test = torch.where(testdata.label == 1)[0].shape[0]
Nnoise_test = torch.where(testdata.label == 0)[0].shape[0]

Nsignal_train = torch.where(traindata.label == 1)[0].shape[0]
Nnoise_train = torch.where(traindata.label == 0)[0].shape[0]

# plot efficiency and FAR vs temperature

In [ ]:
thresholds = np.sort(all_results['threshold'].unique())
temperatures = np.sort(all_results['T'].unique())

In [ ]:
test_eff = {}
for threshold in thresholds:
    test_eff[threshold] = []
    for temp in temperatures:
        test_eff[threshold].append(all_results[(all_results['threshold'] == threshold) & (all_results['T'] == temp)]['test'].item()[0, 0] / Nsignal_test)
    print(test_eff[threshold])
#all_results[all_results['threshold'] == thresholds[1]][['test', 'T']]

test_far = {}
for threshold in thresholds:
    test_far[threshold] = []
    for temp in temperatures:
        test_far[threshold].append(all_results[(all_results['threshold'] == threshold) & (all_results['T'] == temp)]['test'].item()[1, 0] / Nnoise_test)
    print(test_far[threshold])

In [ ]:
figname = '210625_plot_dfcnn_efficiency_vs_noise_temp.png'

sns.set_theme(context='talk', style='whitegrid', palette='mako')
#markers = ['o', 's', 'v', 'd', 'p']
fig = plt.figure(figsize=(16,6))
ax = plt.subplot(1,2,1)

for i, threshold in enumerate(test_eff):
    ax.plot(temperatures, test_eff[threshold], 'o-', label=threshold)
ax.legend(loc=3, title='Threshold')
ax.set_title('Detection Efficiency')
ax.set_xlabel('Noise Temperature (K)')
ax.set_ylabel('Detection Efficiency')
#sns.set_style('whitegrid')

#fig = plt.figure(figsize=(8,6))
ax = plt.subplot(1,2,2)

for i, threshold in enumerate(test_eff):
    ax.plot(temperatures, test_far[threshold], 'o-', label=threshold)

ax.set_title('False Alarm Rate')
ax.set_xlabel('Noise Temperature (K)')
ax.set_ylabel('False Alarm Rate')

ax.legend(loc=2, title='Threshold')

plt.savefig(os.path.join(PLOTPATH, figname))

plt.show()

In [ ]:
all_results